In [334]:
from __future__ import print_function
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
import os
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization

# Instead of training models for each note
- decide which will have notes, for now pick x with one, y with another (train model for this + holds, hands, mines, rolls etc later)
- for each note that will have something, decide what combo it has (train from prev notes (not all 48, pick more relevant ones)) (4 for one note, 6 for 2, 4 for 3) and pick highest class

### TODOS
- test using all info from get_beat_importance instead of aggregate to decide on 1, 2, etc notes (culd also try to predict hold/roll there (would need to train on later beat info as well))

In [338]:
steps_per_bar = 48
class SongFile:
    def __init__(self, key, folder, stepfile, music_file):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        self.note_classes = pd.read_csv('generated_data/{0}_note_classes_generated.csv'.format(key), converters={'0': lambda x: float(x)}).values
        self.notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        self.folder = folder
        self.name = key.split('~')[1]
        self.music_name = music_file
        self.stepfile_name = stepfile
        self.offset = misc[0][0]
        self.beat_length = 60. / misc[1][0]
        self.bpm = misc[1][0]
        self.extension = music_file.split('.')[1]

In [339]:
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
save_files = listdir('data')
save_files_generated = listdir('generated_data')
songs = {}
for song_data in songs_to_use:
    key = song_data[0]
    if '{0}_misc.csv'.format(key) in save_files and '{0}_note_classes_generated.csv'.format(key) in save_files_generated:
        songs[key] = SongFile(key, song_data[1], song_data[2], song_data[3])

In [341]:
beats_to_track = 48
num_classes_one_note = 4
num_classes_two_note = 6
class_map_one_note = {
    '1000': 0,
    '0100': 1,
    '0010': 2,
    '0001': 3
}
class_reverse_map_one_note = ['1000', '0100', '0010', '0001']

class_map_two_note = {
    '1001': 0,
    '0110': 1,
    '1100': 2,
    '1010': 3,
    '0101': 4,
    '0011': 5
}
class_reverse_map_two_note = ['1001', '0110', '1100', '1010', '0101', '0011']

note_types = ['0', '1', 'M', '2', '4', '3']

def get_features_for_row(row):
    return [int(char == target) for target in note_types for char in row]

important_indices = [1, 2, 3, 4, 6, 8, 12, 18, 24, 36, 48]
def get_features(index, features, note_classes):
    indices = [index + beats_to_track - i for i in important_indices]
    return np.array([np.concatenate((note_classes[i], features[i]), axis = 0) for i in indices]).flatten()

In [349]:
X_one_note = []
y_one_note = []
X_two_note = []
y_two_note = []
for key in songs:
    note_classes = np.concatenate((([[1, 0, 0, 0, 0, 0, 0]] * beats_to_track), songs[key].note_classes), axis = 0)
    notes = np.concatenate((([['0000']] * beats_to_track), songs[key].notes), axis = 0)
    if abs(len(note_classes) - len(notes) > 250):
        print ('Lengths dont match for {0}'.format(key))
        print ('{0} vs {1}'.format(len(note_classes), len(notes)))
        continue
    length = min(len(note_classes), len(notes)) - beats_to_track
    features = np.array([get_features_for_row(notes[i][0]) for i in range(-beats_to_track, length)])
    for i in range(length):
        row = notes[i + beats_to_track][0]
        (blank, steps, mines, hold_starts, roll_starts, hold_ends) = [row.count(note_type) for note_type in note_types]
        if steps == 1 and blank == 3:
            X_one_note.append(get_features(i, features, note_classes))
            y_one_note.append(class_map_one_note[row])
            
        if steps == 2 and blank == 2:
            X_two_note.append(get_features(i, features, note_classes))
            y_two_note.append(class_map_two_note[row])

X_one_note = np.array(X_one_note)
y_one_note = np.array(y_one_note)
X_two_note = np.array(X_two_note)
y_two_note = np.array(y_two_note)

In [331]:
def build_model(num_classes):
    model = Sequential()

    model.add(Dense(100, input_dim=275, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                               optimizer='adadelta',
                               metrics=['accuracy'])
    
    return model

In [332]:
y_one_note_hot = np.zeros((len(y_one_note), num_classes_one_note))
y_one_note_hot[np.arange(len(y_one_note)), y_one_note] = 1
y_two_note_hot = np.zeros((len(y_two_note), num_classes_two_note))
y_two_note_hot[np.arange(len(y_two_note)), y_two_note] = 1

In [333]:
y_one_note_model = build_model(num_classes_one_note)
y_one_note_model.fit(X_one_note, y_one_note_hot, nb_epoch=1, batch_size=50)
y_one_note_model.save('models/y_one_note_model.h5')
# Dense 1000: 63s - loss: 1.3934 - acc: 0.2707
# Dense 550: 53s - loss: 1.3903 - acc: 0.2703

Epoch 1/1
73489/73489 [==============================] - 30s - loss: 1.3950 - acc: 0.2699    


In [319]:
y_two_note_model = build_model(num_classes_two_note) .2746 29s
y_two_note_model.fit(X_two_note, y_two_note_hot, nb_epoch=1, batch_size=5)
y_two_note_model.save('models/y_two_note_model.h5')

Epoch 1/1
5225/5225 [==============================] - 17s - loss: 1.8717 - acc: 0.1883    


In [208]:
def normalize_importance(i, beat_importance):
    surrounding_beats = beat_importance[max(i - 12,0):min(i + 12,len(beat_importance))]
    average = sum(surrounding_beats) / float(len(surrounding_beats))
    return beat_importance[i] - (average / 2)

In [211]:
pattern = ['1000', '0100', '0001', '0010', '0100', '1000', '0001', '0010']
def get_output(song):
    predicted_notes = []
    length = len(song.beat_importance)
    beat_importance = [importance[0] for importance in song.beat_importance]
    beat_importance = [normalize_importance(i, beat_importance) for i in range(len(beat_importance))]
    beat_importance_sorted = sorted(beat_importance)
    one_note_cutoff = beat_importance_sorted[int(0.88 * length)]
    two_note_cutoff = beat_importance_sorted[int(0.97 * length)]
    dummy_rows = [row for eigth in pattern for row in [eigth] + ['0000'] * 5]
    features = [get_features(row) for row in dummy_rows]
    for importance in beat_importance:
        if importance < one_note_cutoff:
            prediction = '0000'
        elif importance < two_note_cutoff:
            X_row = [np.array(features[-(beats_to_track - 1):]).flatten()]
            prediction_values = y_one_note_model.predict(np.array(X_row))
            prediction = class_reverse_map_one_note[np.argmax(prediction_values)]
        else:
            X_row = [np.array(features[-(beats_to_track - 1):]).flatten()]
            prediction_values = y_two_note_model.predict(np.array(X_row))
            prediction = class_reverse_map_two_note[np.argmax(prediction_values)]

        predicted_notes.append(prediction)
        features.append(get_features(prediction))
    return predicted_notes

In [210]:
song = songs['In The Groove~Lemmings on the Run']
song = songs['In The Groove~Infection']
song = songs['In The Groove~July']
song.predicted_notes = get_output(song)
step_song(song)

In [114]:
def write_song_header(output_stepfile, song):
    keys = ['TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH', 'SELECTABLE', 'BPMS']
    header_info = {
        'TITLE': song.name,
        'MUSIC': '{0}.{1}'.format(song.name, song.extension),
        'OFFSET': -song.offset,
        'SAMPLESTART': song.offset + 32 * song.beat_length,
        'SAMPLELENGTH': 32 * song.beat_length,
        'SELECTABLE': 'YES',
        'BPMS': '0.000={:.3f}'.format(song.bpm)
    }
    
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, song):
    print("\n//---------------dance-single - J. Zukewich----------------", file=output_stepfile)
    print ("#NOTES:", file=output_stepfile)
    for detail in ['dance-single', 'J. Zukewich', 'Expert', '9', '0.242,0.312,0.204,0.000,0.000']:
        print ('\t{0}:'.format(detail), file=output_stepfile)
    
    for i in range(len(song.predicted_notes)):
        row = song.predicted_notes[i]
        print (row, file=output_stepfile)
        if i % steps_per_bar == steps_per_bar - 1:
            print (",", file=output_stepfile)

    print ("0000;", file=output_stepfile)
    
def step_song(song):
    if song.name + '.sm' in os.listdir(song.folder) and not song.name + '.sm.backup' in os.listdir(song.folder):
        os.rename(song.stepfile_name, song.stepfile_name + '.backup')
            
    output_stepfile=open(song.stepfile_name, 'w')
    write_song_header(output_stepfile, song)
    write_step_header(output_stepfile, song)
    output_stepfile.close()